# Test The Classification Model on the updated dataset

Uses the Random Forest Classifier TF/IDF with the dataset that includes the true Guardian stories

# Imports and Constants

In [1]:
import pickle
import model_helper_functions
import pandas as pd
import numpy as np
import ast

In [2]:
MODEL_PATH = '../models/'
MODEL_FILE_NAME = 'rf_tfidf_plus_guardian_model.sav'

DATA_PATH = '../data/'

In [3]:
DATA_PATH = '../data/'
PRE_PROCESSED_DATA_FILE_NAME = 'guardian_news_dataset_cleaned.csv'
RANDOM_STATE = 42

# Load Model

In [4]:
# using expanded stopwords list from https://gist.github.com/sebleier/554280

gist_file = open(DATA_PATH + "gist_stopwords.txt", "r")
try:
    content = gist_file.read()
    expanded_stopwords = content.split(",")
finally:
    gist_file.close()

In [5]:
expanded_stopwords.remove('via')

In [6]:
expanded_stopwords.remove('eu')
expanded_stopwords.remove('uk')

In [7]:
def lowercase_and_only_expanded_stopwords(doc):
    """Remove stopwords and lowercase tokens"""
    stop_words = expanded_stopwords
    return [token.lower() for token in doc if token.lower() in stop_words]

In [8]:
pipeline = pickle.load(open(MODEL_PATH + MODEL_FILE_NAME, 'rb'))

# Stories to Test the Model With

A story from the Guardian - True

In [9]:
g = "'It's not fair': workers in a poor Mississippi county pay more tax than Trump Clumps of white fluff drift across the highway on the west side of the Mississippi Delta town of Belzoni. Cotton now grows from the beds of former catfish ponds. When catfish ponds in the self-proclaimed Catfish Capital of the World dried up, jobs went with them. Humphreys county, where Belzoni is located, has an unemployment rate double the national level. But it is not entirely a forgotten southern backwater – at least, not to one government agency. Last year ProPublica reported that the Internal Revenue Service audits Humphreys county taxpayers at a higher rate than anywhere else in America. About 12 out of 1,000 tax returns are audited each year in the county, according to the ProPublica analysis. That’s 53% higher than the national average and raises the question of why such a place is the subject of such IRS attention, rather than the haunts of millionaires and billionaires, like Manhattan and its one-time resident Donald Trump. But in the most heavily audited county in the US, the recent shock news that Trump paid just $750 in federal income taxes in 2016 and 2017 was no surprise, said Joe Jackson, who was elected mayor of Belzoni last month. "

Two new stories from Reuters - True

In [10]:
r = 'Thousands marched to the U.S. Supreme Court in Washington on Saturday to commemorate the late Justice Ruth Bader Ginsburg and protest President Donald Trump’s rush to push through Amy Coney Barrett as her replacement. The U.S. Senate Judiciary Committee has scheduled an Oct. 22 vote on the nomination of Barrett, a conservative appellate judge, over objections from Democrats that the confirmation process comes too close to the Nov. 3 presidential election. More than 26 million Americans have already cast their ballots for who they want to sit in the White House for the next four years, Trump or his Democratic rival Joe Biden. Demonstrators at the Women’s March said they were angry that Republicans appear ready to confirm Barrett’s nomination so close to Election Day after refusing to move forward Merrick Garland, the pick of former President Barack Obama, a Democrat, more than six months ahead of the 2016 election. “The fact of the matter is that we are powerful and they are afraid,” said Sonja Spoo, the director of the reproductive rights campaigns at UltraViolet, a feminist advocacy group, one of the speakers at the protest. “They are on the ropes and they know it and we are about to give the knock-out punch.” Ginsburg, a liberal champion of women’s rights, died on Sept. 18. Prudence Sullivan, 49, from Lake in the Hills, Illinois, near Chicago, and her sister Kelli Padgett, 47, from Jacksonville, Florida, flew in to join what they described as an energizing and empowering event. “We’ve had losses from COVID and we’ve clashed with family members over racism, Black Lives Matter,” Sullivan said. “So this is something where I can put my money where my mouth is.” Sullivan said she and her husband, an IT expert, were looking at options for moving overseas if Trump was reelected. The protesters marched through downtown Washington to the Supreme Court steps. Hundreds of marches and demonstrations were planned at city halls, parks and monuments across the country. In confirmation hearings this week, Barrett side-stepped questions about presidential powers, abortion, climate change, voting rights and Obamacare, saying she could not answer because cases involving these matters could come before the court. If Barrett takes a seat on the Supreme Court, conservatives would have a 6-3 majority.'

In [11]:
r2 = 'Guinea votes in election, with president seeking a third term Voters in Guinea went to the polls on Sunday as octogenarian President Alpha Conde sought to extend his decade in power after pushing through a new constitution that allowed him to run for a third term, sparking months of violent protests. At least 50 people have been killed over the past year during demonstrations against the new constitution, Amnesty International said, and violence erupted repeatedly during campaigning in recent weeks. In the capital, Conakry, dozens of voters queued quietly at a polling station in a primary school whose exterior wall was plastered with posters of Conde. “I hope that things will happen in peace,” said Mariama Camara, a housewife. “I voted for change,” said Mamadou Diallo, a trader. While no reliable opinion polls are available, many political analysts expect Conde to prevail after he won overwhelming approval for the new constitution in a referendum in March - although that vote was boycotted by the opposition. Conde, 82, faces 11 challengers, including his long-time rival Cellou Dalein Diallo. Diallo, a former prime minister who finished runner-up to Conde in elections in 2010 and 2015, has warned about fraud and said he will challenge any irregularities. Conde, who has described the constitutional reform as fair and democratic, says he needs more time to finish major mining and infrastructure projects in the West African country. “Guinea cannot develop if there is not peace, security and unity. We do not want violence,” Conde said after casting his ballot at the Boulbinet primary school in Conakry. “Those who want to challenge the results must do so within a legal framework, with recourse to the constitutional court,” he said, dressed in white and flanked by bodyguards. Guinea has made progress developing its mineral riches, including bauxite and iron ore, and gross domestic product has doubled under Conde’s presidency. But many Guineans complain the mining boom has not ended frequent power cuts and unemployment. The national election commission said on Saturday provisional results would be published within three days after it receives the last polling station tally. The constitutional court will then have eight days to declare a winner. The United Nations has warned about divisive appeals to ethnic affiliations on the campaign trail. Conde and Diallo both draw much of their support from their respective ethnic communities. Guinea has been plagued by sporadic political unrest since independence from France in 1958, often fuelled by ethnic tensions. Pro-democracy activists are concerned that the election - and a presidential vote due later this month in neighboring Ivory Coast - may damage democracy in a region that had previously won praise for its leaders’ adherence to term limits. Ivorian President Alassane Ouattara, who first won power in 2010, is also seeking re-election, having argued that a new constitution in 2016 reset the clock on the two-term limit.'

A story from the True dataset (also from Reuters)

In [12]:
a = '(Reuters) - U.S. President-elect Donald Trump will name former Georgia Governor Sonny Perdue as his nominee for secretary of agriculture on Thursday, a senior transition official said on Wednesday. Here is a list of Republican Trump’s selections for top jobs in his administration.  NOTE: Senate confirmation is required for all the posts except national security adviser and White House posts. Tillerson, 64, has spent his entire career at Exxon Mobil Corp, where he rose to chairman and chief executive officer in 2006. A civil engineer by training, the Texan joined the world’s largest publicly traded energy company in 1975 and led several of its operations in the United States as well as in Yemen, Thailand and Russia. As Exxon’s chief executive, he maintained close ties with Moscow and opposed U.S. sanctions against Russia for its incursion into Crimea. Mnuchin, 54, is a successful private equity investor, hedge fund manager and Hollywood financier who spent 17 years at Goldman Sachs Group Inc before leaving the investment bank in 2002. He assembled an investor group to buy a failed California mortgage lender in 2009, rebranded it as OneWest Bank and built it into Southern California’s largest bank. Housing advocacy groups criticized OneWest for its foreclosure practices, accusing it of being too quick to foreclose on struggling homeowners. Mattis is a retired Marine general known for his tough talk, distrust of Iran and battlefield experience in Iraq and Afghanistan. A former leader of Central Command, which oversees U.S. military operations in the Middle East and South Asia, Mattis, 66, is known by many U.S. forces by his nickname, “Mad Dog.” He was rebuked in 2005 for saying: “It’s fun to shoot some people.” Sessions, 70, was the first U.S. senator to endorse Trump’s presidential bid and has been a close ally since. Son of a country store owner, the lawmaker from Alabama and former federal prosecutor has long taken a tough stance on illegal immigration, opposing any path to citizenship for undocumented immigrants. Coats, 73, served as U.S. senator from Indiana from 1989 to 1999 and again from 2011 to 2017, and was U.S. ambassador to Germany from 2001 to 2005. He previously served as U.S. representative from Indiana’s 4th Congressional District. Zinke, 55, a first-term Republican U.S. representative from Montana and a member of the House subcommittee on natural resources, has voted for legislation that would weaken environmental safeguards on public lands. He has taken stances favoring the coal industry, which suffered during the Obama administration. The League of Conservation Voters, which ranks lawmakers on their environmental record, gave Zinke an extremely low lifetime score of 3 percent. Ross, 79, heads the private equity firm WL Ross & Co. Forbes has pegged his net worth at about $2.9 billion. A staunch supporter of Trump, Ross helped shape the Trump campaign’s views on trade policy. He blames massive U.S. factory job losses on the North American Free Trade Agreement (NAFTA) with Canada and Mexico, which went into force in 1994, and the 2001 entry of China into the World Trade Organization.      Lighthizer, 69, served as deputy U.S. trade representative during the Reagan administration in the 1980s and has since spent nearly three decades as a lawyer representing U.S. companies in anti-dumping and anti-subsidy cases, currently with the law firm Skadden, Arps, Slate, Meagher and Flom. A harsh critic of China’s trade practices, Lighthizer in 2010 told Congress that U.S. policymakers should take a more aggressive approach in dealing with the Asian country. Puzder, chief executive officer of CKE Restaurants Inc [APOLOT.UL], which runs the Carl’s Jr. and Hardee’s fast-food chains, has been a vociferous critic of government regulation of the workplace and the National Labor Relations Board. Puzder, 66, has argued that higher minimum wages would hurt workers by forcing restaurants to close and praises the benefits of automation, so his appointment is likely to antagonize organized labor.  U.S. Representative Price, 62, is an orthopedic surgeon who heads the House Budget Committee. A representative from Georgia since 2005, Price has criticized Obamacare and has championed a plan of tax credits, expanded health savings accounts and lawsuit reforms to replace it. He is against abortion. Former Georgia Governor Perdue, 70, served on Trump’s agricultural advisory committee during the campaign. Perdue, a Republican, was elected to two terms as governor, serving from 2003 to 2011. Before that, he served in the state Senate representing a rural swath of the state about 100 miles (160 km) south of Atlanta. After finishing his second term as governor, Perdue founded Perdue Partners, a global trading firm that consults and provides services for companies looking to export products. Shulkin, 57, currently is under secretary for health at the Department of Veterans Affairs, putting him in charge of the country’s largest healthcare system. Shulkin, a physician, was chosen by Democratic President Barack Obama for the under secretary post in 2015. He has spearheaded an effort to cut waiting times for care at VA medical centers. Trump promised during the campaign to improve medical care for veterans. Shulkin would be the first VA secretary who had not served in the military. Carson, 65, is a retired neurosurgeon who dropped out of the Republican presidential nominating race in March and threw his support to Trump. A popular writer and speaker in conservative circles, Carson had been reluctant to take a position in the incoming administration because of his lack of experience in the federal government. He is the first African-American picked for a Cabinet spot by Trump. Chao, 63, was labor secretary under President George W. Bush for eight years and the first Asian-American woman to hold a Cabinet position. She is a director at Ingersoll Rand Plc, News Corp and Vulcan Materials Co. She is married to U.S. Senate Majority Leader Mitch McConnell, a Republican from Kentucky. Perry, 66, is an addition to the list of oil drilling advocates skeptical about climate change who have been picked for senior positions in Trump’s Cabinet. The selections have worried environmentalists but cheered an oil and gas industry eager for expansion. Perry, who ran unsuccessfully for the 2012 Republican presidential nomination and also briefly ran for president in 2016, would be responsible for U.S. energy policy and oversee the nation’s nuclear weapons program.  DeVos, 59, is a billionaire Republican donor, a former chair of the Michigan Republican Party and an advocate for the privatization of education. As chair of the American Federation for Children, she has pushed at the state level for vouchers that families can use to send their children to private schools and for expansion of charter schools. The final leadership role of Kelly’s 45-year military career was head of the U.S. Southern Command, responsible for U.S. military activities and relationships in Latin America and the Caribbean. The 66-year-old retired Marine general differed with Obama on key issues and has warned of vulnerabilities along the United States’ southern border with Mexico. Priebus recently was re-elected to serve as Republican National Committee chairman but will give up that job to join Trump in the White House, where the low-key Washington operative could help forge ties with Congress to advance Trump’s agenda. Priebus, 44, was a steadfast supporter of Trump during the presidential campaign even as the party fractured amid the choice. ENVIRONMENTAL PROTECTION AGENCY ADMINISTRATOR: SCOTT PRUITT An ardent opponent of Obama’s measures to stem climate change, Oklahoma Attorney General Pruitt, 48, has enraged environmental activists. But he fits in with the president-elect’s promise to cut the agency back and eliminate regulation that he says is stifling oil and gas drilling. Pruitt became the top state prosecutor for Oklahoma, which has extensive oil reserves, in 2011 and has challenged the EPA multiple times since. U.S. Representative Mick Mulvaney, 49, a South Carolina Republican, is a fiscal conservative. He was an outspoken critic of former House of Representatives Speaker John Boehner, who resigned in 2015 amid opposition from fellow Republicans who were members of the House Freedom Caucus. Mulvaney was first elected to Congress in 2010. Haley, 44, has been the Republican governor of South Carolina since 2011 and has little experience in foreign policy or the federal government. The daughter of Indian immigrants, she led a successful push last year to remove the Confederate battle flag from the grounds of the South Carolina state capitol after the killing of nine black churchgoers in Charleston by a white gunman. McMahon, 68, is a co-founder and former chief executive of the professional wrestling franchise WWE, which is based in Stamford, Connecticut. She ran unsuccessfully as a Republican for a U.S. Senate seat in Connecticut in 2010 and 2012 and was an early supporter of Trump’s presidential campaign. U.S. Representative Pompeo, 53, is a third-term congressman from Kansas who serves on the House of Representatives Intelligence Committee, which oversees the CIA, National Security Agency and cyber security. A retired Army officer and Harvard Law School graduate, Pompeo supports the U.S. government’s sweeping collection of Americans’ communications data and wants to scrap the nuclear deal with Iran. Walter “Jay” Clayton is a New York-based attorney who advises clients on major Wall Street deals, specializing in public and private mergers and acquisitions and capital-raising efforts. His past clients have included Alibaba Group Holding Company, Oaktree Capital Group and big banks.    Retired Lieutenant General Flynn, 58, was an early Trump supporter and serves as vice chairman on his transition team. He began his Army career in 1981 and was deployed in Afghanistan and Iraq. Flynn became head of the Defense Intelligence Agency in 2012 under Obama but retired a year earlier than expected, according to media reports, and became a fierce critic of Obama’s foreign policy. Tom Bossert, 41, who worked as deputy homeland security adviser to former President George W. Bush, will serve as the assistant to the president for homeland security and counterterrorism. He currently runs a risk management consulting firm and has a cyber risk fellowship with the Atlantic Council think tank in Washington. Cohn, 56, president and chief operating officer of Goldman Sachs, had widely been considered heir apparent to Lloyd Blankfein, CEO of the Wall Street firm. Trump hammered Goldman and Blankfein during the presidential campaign, releasing a television ad that called Blankfein part of a “global power structure” that had robbed America’s working class. Navarro, 67, has suggested a stepped-up engagement with Taiwan, including assistance with a submarine development program. A professor at the University of California, Irvine, who advised Trump during the campaign, Navarro argued that Washington should stop referring to the “one China” policy, but stopped short of suggesting it should recognize Taipei: “There is no need to unnecessarily poke the Panda.” Viola, 60, is a graduate of the U.S. Military Academy at West Point and a U.S. Army veteran who served in the famed 101st Airborne Division. He founded high-frequency trading firm Virtu Financial Inc and served as chairman of the New York Mercantile Exchange, where he began his financial services career. After the Sept. 11, 2001, al Qaeda attacks on New York and Washington, Viola helped found the Combating Terrorism Center at West Point. He is an owner of the Florida Panthers ice hockey team. CHIEF WHITE HOUSE STRATEGIST, SENIOR COUNSELOR: STEVE BANNON The former head of the conservative website Breitbart News came aboard as Trump’s campaign chairman in August. A rabble-rousing conservative media figure, he helped shift Breitbart into a forum for the alt-right, a loose confederation of those who reject mainstream politics and includes neo-Nazis, white supremacists and anti-Semites. His hiring signals Trump’s dedication to operating outside the norms of Washington. As White House chief of staff, Bannon, 63, will serve as Trump’s gatekeeper and agenda-setter. '

Two Stories from the NYT - True

In [13]:
nyt_title = 'Flush With Cash, Biden Eclipses Trump in War for the Airwaves'

In [14]:
nyt_text =  'President Trump is being vastly outspent by Joseph R. Biden Jr. in television advertising in the general election battleground states and elsewhere, with the former vice president focusing overwhelmingly on the coronavirus as millions of Americans across the country begin casting early votes. Mr. Biden has maintained a nearly 2-to-1 advantage on the airwaves for months. His dominance is most pronounced in three critical swing states — Michigan, Pennsylvania and Wisconsin — where he spent about $53 million to Mr. Trump’s $17 million over the past month largely on ads assailing the president’s handling of the virus as well as the economy and taxes, according to data from Advertising Analytics, an ad tracking firm. In Pennsylvania alone, Mr. Biden ran 38 different ads during a single week this month, a sign of how comprehensive his effort there has been. The president’s ad strategy, in turn, reflects the challenges facing both his campaign finances and Electoral College map. He has recently scaled back advertising in battleground states like Ohio and Iowa and, until this past week, slashed ads in Michigan and Wisconsin, despite being behind in polls. And Mr. Trump is having to divert resources to hold onto Republican-leaning states like Arizona and Georgia. Mr. Trump spent less on ads in 2016, too, but he went on to narrowly capture critical states anyway and prevail over Hillary Clinton. Back then he relied heavily on huge rallies and live cable news coverage to get his message out, and he got extensive airtime for his attacks on Mrs. Clinton. This time around, his rallies have been fewer and smaller because of the pandemic and his own virus infection; the events have gotten less cable coverage; and he has had a hard time making attacks stick on Mr. Biden. In many ways, the advertising picture reveals how the pandemic has upended the 2020 race. With in-person campaigning sharply limited, the traditional advantages built by a ground game in battleground states have largely been replaced by the air cover provided by advertising. More than $1.5 billion has been spent on the presidential race alone; by contrast, $496 million was spent on ads in just the presidential race by this point in the 2016 race. Onscreen, voters are inundated with imagery of the pandemic — hazmat suits and shuttered businesses — as well as scenes from protests, both peaceful and violent. Roughly 80 percent of the Trump campaign’s ads have been either negative or what’s called a contrast ad, a mix of criticism of the opponent and self-promotion. Of those, 62 percent were all-out attacks. For Mr. Biden, about 60 percent of campaign ads have been negative or contrast, with just 7 percent outright negative. This inundation can be dizzying to viewers in swing states. In Phoenix, during the Spanish language broadcast of “Exatlón,” a popular reality TV show, a Biden ad that hails Senator Kamala Harris s record with the Latino community and that shows her peacefully marching in the streets is followed by a Trump ad decrying Mr. Biden as “radical,” with scenes of violent clashes from protests. “Exatlón” runs an average of 15 ads from the Trump or Biden campaigns per broadcast. In Philadelphia, fans of “Judge Judy” had to sit through an average of five political ads per half-hour of broadcast, with the Biden and Trump campaigns offering dueling arguments on criminal justice reform. The increasingly lopsided nature of the TV ad wars is part of an overall narrative that shows a persistent, if slight, advantage for the former vice president. Mr. Biden has built steady, single-digit leads in battleground polls with a campaign message of resolve and a pledge to control the pandemic — points reflected in his ads. Mr. Trump, who has repeatedly shifted his messaging, has played more defense on the airwaves. The result is a reversal of fortunes in a campaign where it seemed Mr. Trump had built an unstoppable cash machine early on and, in the words of his former campaign manager, had a “death star” of political advertising ready for the final push. Having been outspent by roughly $124 million on the airwaves since May, Mr. Trump has largely ceded the ad wars to Mr. Biden. “The candidates almost always try to match what their opponent is doing because it’s like an arms race,” said Lynn Vavreck, a politics professor at the University of California, Los Angeles. “They understand that allowing them to get ahead has consequences.” For months, the Trump campaign held a significant advantage on digital advertising, though recently the Biden campaign has essentially matched the Trump footprint online, reaching a rough parity on Google and Facebook over the past 30 days, at about $50 million on the platforms for each campaign. Of course, the overall effectiveness of political advertising varies widely from race to race, and is likely to have less impact in winning over voters when an electorate is exceptionally polarized, as it is in 2020. But given the slowdown of normal operations, campaigns have relied on television ads this cycle to simply remain in the daily conversation. Both campaigns are spending by far the most money and airing the most ads in Florida. It’s a must-win for Mr. Trump; if Mr. Biden prevails there, political strategists believe it would signal a clear path to victory. Mr. Biden has spent about $14 million more on broadcast television and cable ads there than Mr. Trump. The latest polling average calculated by The Times’s Upshot shows Mr. Biden with a four-point advantage. In Florida, perhaps no markets offer a greater indication of both campaigns  strategies than Miami-Fort Lauderdale, home to Mr. Biden s biggest base of support in the state, and the Tampa-St. Petersburg media market, which includes Pinellas County, a populous county that switched from blue to red in 2016. Tampa is one of the few markets where Mr. Trump has been outspending Mr. Biden over the past 30 days, as his campaign tries to hold onto support with a largely negative campaign focused on Mr. Biden s tax proposals. But in Miami, Mr. Biden is the dominant force on the airwaves. Democratic operatives view their path to winning Florida through a massive turnout in Miami, the state’s most diverse city, where Democrats have a huge advantage. Mr. Biden has been airing two biographical spots — one in Spanish and one in English, each with near-equal money behind them — that focus on his personal history benefiting from health care, and his fights to expand that access through the Affordable Care Act. “That had extraordinary potency in 2018,” said Roy Temple, a Democratic ad maker, referring to the health care ads and messaging that helped Democrats take back the House two years ago. “If anything, Covid added to the potency of that, because you’ve got literally millions of people who now have a new pre-existing condition.” But while the Biden campaign is looking to Florida for a knockout punch, it has also steadily been landing body blows in the former “blue wall” states of Pennsylvania, Michigan and Wisconsin that Mr. Trump won in 2016. Take Milwaukee, where Mr. Biden has been owning the airwaves at a nearly 5-to-1 spending advantage over the past 30 days. While it is the Democratic anchor of the state and where Mr. Biden needs to turn out a large part of his base, the Milwaukee media market includes Kenosha and Racine, two of the 10 biggest counties in the state and ones that Mr. Trump flipped to red in 2016. Now, voters in the two counties, as well as in Milwaukee, have seen an average of 139 Biden ads each day for the past week, and just 17 Trump ads. The Biden campaign has been running a mix of ads that are heavy on personal biography and that contrast his vision for controlling the pandemic with Mr. Trump’s handling of the virus. “When you get a candidate in Biden’s position, you no longer are in a situation where you really are trying to knock the other guy down,” said Tad Devine, a Democratic ad strategist who ran the ad campaigns of both John Kerry and Al Gore. “You are in a situation where you must offer fundamental reassurance.” In Detroit and Philadelphia, the ad disparity is similarly stark, with Mr. Biden more than doubling Mr. Trump’s spending in both cities. The Biden campaign has been making an effort to energize Black voters, with ads featuring the actor Samuel L. Jackson and young Black millennials from Flint, Mich., discussing the importance of voting. But while the traditional swing states occupy most of the attention, Mr. Devine says the Biden campaign’s dominance in Arizona is among the biggest indicators of his campaign’s strength. “If you can move into the other guy’s terrain and force them either to defend it or actually make a play to take it away from them,” Mr. Devine said, “that’s the single most aggressive play there is in a presidential campaign — to move against an opponent in turf that belongs to them.” The Biden campaign’s large investment in Arizona has forced the Trump campaign to move some money around as well, adding $5.7 million more to its ad reservations in Arizona since Aug. 30. Forcing the Trump campaign to spend more in Arizona, a Republican state that hasn’t voted for a Democratic presidential candidate since 1996, is most likely a factor that has forced the campaign to draw down its advertising in states like Ohio and Iowa, two traditional battleground states that the campaign won handily in 2016 and is hoping to do again this year. But it has also meant that the Trump campaign has significantly reduced spending in Minnesota, a traditionally blue state that Republicans had hoped would be in play in 2020. Mr. Biden has had double-digit leads in most polls there. The Trump campaign claims that all of its ad decisions have been strategic, and not budget-forced. “Television ads are a small piece of the voter outreach puzzle and the Trump campaign has perfected the art of utilizing them in the most strategic, surgical way possible,” said Samantha Zager, the deputy press secretary for the Trump campaign. “It makes no sense to run TV ads in states we know we’re going to win, and in other states, they’re a useful tool to reach the right voters with the right message.”  '

In [15]:
nyt_title_2 = "Biden Says He Gets Places Like Scranton. Some in Town Aren’t So Sure."

In [16]:
nyt_text_2 = 'SCRANTON, Pa. — Despite it all, Gabriel Perez, the Empanada King of Scranton, is still hopeful about America. In the first few months of the pandemic, “it was scary, business dropped for a little while,” he said. But now, more people are ordering delivery from his small shop where he has been serving steaming beef and chicken empanadas to go since 2016. Last month he invested in a renovation of his kitchen, with new equipment and a fresh coat of paint. Mr. Perez did not vote for Donald J. Trump in 2016. He did not vote at all. But he does not dislike him. Mr. Trump’s book “The Art of the Deal” gave him “guidelines for how to run my business: No matter how many times you make a mistake, just keep going.” And while he does not think he will vote for President Trump this year, he is not sold on former Vice President Joseph R. Biden Jr. either. “The Democrats and Republicans are both lost,” he said a few weeks ago in his shop, a Biden ad on mute on the television on the wall. “They are not giving solutions. They just want you to pick their side.” In the final weeks of the campaign, Mr. Biden has made Scranton, his hometown, a major part of his closing pitch. “I really do view this campaign as a campaign between Scranton and Park Avenue,” he said at a CNN event in town last month. Embedded in Mr. Biden’s shorthand is that he can win back the paradigmatic Scranton voter: white, working class, disaffected by Democrats. But Scranton is no longer the dying coal town of Mr. Biden’s youth. It is both more racially diverse and prosperous. In more than two dozen interviews the week of Mr. Biden’s visit, few voters were particularly enthusiastic about his candidacy, despite his personal roots, but about half said they probably would vote for him anyway. Voters who abandoned the Democratic Party in 2016 said they planned to vote for Mr. Trump again this year. Some people said they were so fed up with politics that they were not going to vote at all. Others expressed annoyance at what they said was Mr. Biden’s habit of making Scranton into a kind of blue-collar cartoon. At the town-hall-style event, held six miles from downtown in a stadium parking lot, Mr. Biden said that not many people in Scranton owned stock. “Frankly, it was insulting,” said Frances Keating, 74, a retired accountant who has lived in Scranton most of her life. “He’s using Scranton as a prop.” Still, she said she planned to vote for Mr. Biden because “Trump is a monster.” Scranton has become a symbol for Democrats’ lost dreams in 2016, when working-class voters abandoned the party in droves. The city itself is blue. But the surrounding county, Lackawanna, and a neighboring one, Luzerne, had the second- and third-largest swings toward Mr. Trump of any county with more than 100,000 voters in the United States. The surge was enough to cover his 44,000-vote victory in Pennsylvania. Mr. Trump is trailing in the state by seven percentage points, but the enthusiasm he enjoys among many ancestral Democrats in Scranton highlights the challenges Mr. Biden still faces in a state regarded by both parties as a must-win next month. Kim Anzelmi, a former meat inspector, was watching television after dinner in a suburb of Scranton last month when Mr. Biden flashed on the screen. “Mr. President, do your job,” Mr. Biden said. Ms. Anzelmi scoffed. She said that she was tired of hearing from progressives that she had privilege because she was white and that she feared a Biden presidency would only give them more power. “I put myself through college,” said Ms. Anzelmi, who is 55 and whose vote for Mr. Trump in 2016 was the first she had ever cast for president. “I was a security guard at Sears. I worked in meat plants where I was the only woman. Now you tell me I’m entitled?” That Mr. Biden is from this area did not matter. Ms. Anzelmi plans to vote for Mr. Trump again. But in a sign of how complicated politics have become within families, her husband — an immigrant from Uruguay who got his citizenship in 2009 but has never voted — likes Mr. Biden and said he thought he might vote for him. Mr. Trump “is crazy,” he said, sitting in an armchair near his wife. “He speaks too much.” He asked his wife why she liked him. “I agree with Trump because I got screwed,” she said. “But nobody does nothing for you,” he said. “It’s going to be the same.” A saw-shaped spot on the map whose coal mines drew immigrants from Europe in the 19th and early 20th centuries, Scranton has a population that peaked in the 1930s and declined sharply in the following decades. In the early 2000s, the economy began to pick up. Located at the intersection of several major Interstate highways, logistics companies and warehouses began to open up. The economic engines that powered revivals in places like Pittsburgh — education and health care — have grown too. Colleges have expanded dramatically. A new medical school has opened. Demographics have changed too. Hispanics are now about 14 percent of the population in Scranton. In Hazleton, a city in Luzerne County, they make up more than half. Scranton now has two Hindu temples and Indian grocery stores that serve South Asians working in the hospitals and at a nearby T.J. Maxx warehouse. But immigrants in Scranton do not all default to the Democratic Party. The day after Mr. Biden’s visit, Dipen Vyas was standing outside his Indian food supply store with two friends. All of them were Indian immigrant business owners and all had voted for Mr. Trump in 2016. They liked that he was a businessman, and that he had promised to be tough on illegal immigration. But they could not decide whether to vote for Mr. Trump again. On the one hand, Indians who had been waiting years for permanent residence were now being rejected. On the other, the president seemed to have struck up a strong relationship with Prime Minister Narendra Modi of India. But they liked Mr. Biden too. He was from Scranton. And he had already been vice president, so he knew the ropes. “I’m going to toss the coin,” Mr. Vyas said. “Heads or tails.” On Scranton’s South Side, where new arrivals from Mexico, Central America and the Caribbean are now moving into houses that used to belong to immigrants from Southern and Eastern Europe, Mr. Perez at the empanada shop, Papi’s Kitchen, was reflecting on the political decision before him. In 2016, Mr. Perez preferred Bernie Sanders. Health care for everyone and free tuition made sense to him. But he did not vote after Mr. Sanders lost the Democratic nomination. That year, he opened his empanada shop in what used to be a pizza takeout place. '

Story from Breitbart - Fake

In [17]:
bb_title = "Biden Campaign Manager Warns Race Will ‘Come Down to the Wire’"

In [18]:
bb_text = 'Former Vice President Joe Biden’s campaign manager is warning supporters the upcoming general election will “come down to the wire” and many of the critical swing states “are functionally tied.” On Saturday, the Hill published excerpts of a memo that Jennifer O’Malley Dillon, the Democrat nominee’s campaign manager, sent to supporters and activists urging them not to take Biden’s lead in the polls as an assurance of victory. “We cannot become complacent because the very searing truth is that Donald Trump can still win this race, and every indication we have shows that this thing is going to come down to the wire,” Dillon wrote, adding Democrats did not want to repeat the mistakes of 2016 when turnout dropped because polls inaccurately showed former Secretary of State Hillary Clinton winning in a landslide. O’Malley Dillon further cautioned supporters in the key 2020 battlegrounds the “race is far closer than some of the punditry we’re seeing on Twitter and on TV would suggest.” “While we see robust leads at the national level, in the states we’re counting on to carry us to victory like Arizona and North Carolina we’re only up by three points.” she wrote. “We also know that even the best polling can be wrong, and that variables like turnout mean that in a number of critical states we are functionally tied — and that we need to campaign like we’re trailing.” O’Malley Dillon’s suggestion the race is “far closer” than many believe comes only days after she took to social media to implore supporters and volunteers to ignore polls and continue investing time and resources in the race. While some have suggested the Biden campaign is simply attempting to keep voters energized until Election Day, there are also some indicators that the presidential contest is tightening. Polls in recent days have shown Biden leading the incumbent, President Donald Trump, by narrow margins in states like Arizona and Wisconsin. Meanwhile, in states like North Carolina and Florida, polls show the race tied. Compounding problems for Democrats is that GOP voter registration has narrowed the voter registration gap in numerous swing states. In Pennsylvania, which Trump won narrowly four years ago, the GOP has seen its support grow substantially. According to a voter registration report released in June by Pennsylvania’s Department of State, Republicans added 258,705 new registrants between December 2015 and December 2019. Over the same period, in comparison, Democrats only gained 85,779 new registrants. The GOP’s success has narrowed the Democrats’ voter advantage in Pennsylvania to just slightly more than 710,000 voters. Although that split seems large, Democrats had an even larger registration advantage in 2016 when Trump bested former Secretary of State Hillary Clinton in the state by just 44,292 votes out of more than six million ballots. Trump’s margin of victory, narrow though it was, made him the first Republican to carry Pennsylvania since President George H.W. Bush’s 1988 landslide. Recent polls show the GOP’s investment in voter registration might be paying off. A survey released by the Trafalgar Group on Tuesday found Trump and Biden statistically tied in the Keystone State.'

Story from the Fake dataset

In [19]:
b = ' Funny how secrets travel. I d start to believe, if I were to bleed.    Lyrics written by David Bowie from the album OutsideShawn Helton 21st Century Wire In the days before the US presidential election in November 2016, Wikileaks revealed a new string of emails connected to Hillary Clinton s 2016 campaign chairman, John Podesta. Online interpretations of the leaked Podesta email exchanges subsequently led to a digital firestorm on social media, producing an intense conspiratorial  open source investigation  still in search of its smoking gun. However, the phenomenon now known by its hashtag #Pizzagate has since set a number of tertiary events into motion. Is there a real story hidden amongst the debris of this online explosion?Is it possible that  open source investigators  were being led into a virtual cul-de-sac and thus missing the real story?More importantly, could people be playing into the hands of the very establishment they seek to depose?Let s investigate   SHOCK & AWE    Over the past month, the DC-based restaurant Comet Ping Pong, along with Besta Pizza (to a lesser degree Austin s East Side Pies) has been the subject of a peculiar internet-based conspiracy. (Photo Illustration Shawn Helton of 21WIRE)CONTEXT: Podesta Emails, False Hacks & ConspiracyWhile many of the leaked Podesta emails (including the really important one admitting that US partners Qatar and Saudi Arabia are giving funding and  logistical support  of ISIS) caused a major stir in the run-up to the 2016 Election, few Wikileaks releases have attracted more attention then the Podesta emails associated with the internet-based conspiracy dubbed  Pizzagate. In fact, since the release of the so-called  pizzagate  emails, a bundle of leaks and media reports associated with the Clintons political pay-for-play allegations, as well as the shocking admission regarding voter fraud from disgraced Democratic National committee consultant Robert Creamer prior to the 2016 presidential election, have all but disappeared from the mainstream narrative. Very convenient, if you are part of the Democratic Party establishment.Remember that all of this was taking place right after the mainstream media along with the White House, set the stage for a massive PR push implicating Russia as a disrupting force for evil in the US presidential election   allegedly through Kremlin-sponsored hacking of US democratic institutions and through  sophisticated Russian propaganda. In August, the western media s claims against Russia hit overdrive, when the  New York Times s Moscow bureau was the target of an attempted cyber attack this month. But so far, there is no evidence that the hackers, believed to be Russian, were successful. Flash forward to September here and here, as well as early October in the lead up to President Obama s Russian Hack decree, the Clinton-friendly outlet, the NY Times, had all but solidified the Russian cyber-hack claims   but once again, without any definitive evidence or proof.Throw in another American  October surprise  in the form distributed denial of service (DDoS) attacks, renewed calls for the Stop Online Piracy Act (SOPA) internet, led to so-called cyber experts blaming the largest  hack on America  because of the inability to pass SOPA   in effect, reigniting the previously stalled SOPA in Washington DC, along with all of its draconian copyright law positions   all perfectly timed as mainstream reports regarding  fake news suddenly became all the rage across all forms of media.The media clips on YouTube below illustrate the kind of duplicitous double speak regarding the  election rigging  that has become a hallmark of the Obama administration and establishment Washington Following 2016 s US presidential election cycle, a Russian election-hack-theory gained even more media traction, orbiting the political realm like a low-hanging fruit (propelled by President Obama and now the CIA, both conflicting with the FBI s assessment in October), dovetailing with the first grumblings that  fake news  had somehow decided the US presidential election in favor of one candidate, Donald Trump. These are serious accusations that have still failed to yield any solid forensic results from the intelligence community.Is this mere political gamesmanship, or is there more at work?Could this all be covering up and diverting the public away from some other explosive events?  LOLITA EXPRESS    In 2008, Jeffery Epstein was convicted of soliciting an underage girl for prostitution (Image Source: miamibeachadvocate.com)As internet researches battle with mainstream media over allegations in #Pizzagate, another potentially much bigger story became buried under the digital avalanche. If this is indeed the case, then who benefits?The  Lolita Express  ScandalSeemingly even more unsettling and disturbing in 2016, was the vastly underreported  Lolita Express  scandal swirling around the Clinton Campaign.In 2010, Jeffery Epstein, a billionaire hedge fund mogul who is also a member of the Trilateral Commission and the Council on Foreign Relations (as well as playing a role in the formation of the Clinton Global Initiative), effectively walked free even though a 53-page indictment could have landed him in prison for 20 years according to reports. Epstein s  Gatsbyesque  persona once described by New York magazine, has long since given way to a new profile as a convicted sex-offender  a superstar who was now condemned by a litany of legal troubles due to reports of ghastly serial abuse during the past decade.However, during this year s election cycle, coverage concerning new details associated with Epstein s private island, Little St. James, and allegations of sex slave entertainment enterprise there, was virtually buried by the US corporate media.The drop in wall-to-wall coverage from the tongue-tied mainstream media was most likely due to Epstein s close affiliation to former President Bill Clinton, who according flight manifests and other supporting evidence, took  at least 26 trips aboard the  Lolita Express    even apparently ditching his Secret Service detail for at least five of the flights, according to records obtained by FoxNews.com. It doesn t stop there either. Other high-profile names linked to the Epstein case included former Democratic governor Bill Richardson, Kurt Cobain widow Courtney Love, prominent celebrity lawyer, Israeli advocate and Harvard Law professor Alan Dershowitz, British royal Prince Andrew, former Israeli prime minister Ehud Barak, and even president-elect Donald Trump.  THE BLACK BOOK     Epstein flight records and  a little black book  paint a very intriguing picture. (Image Source: Gawker)Interestingly, the Epstein case was covered heavily by Gawker   a news outlets which is now facing bankruptcy following a defamation case won by Hulk Hogan (Terrence Gene Bollea) over the controversial publication of the wrestling icon s sex tape. Hogan s legal victory  took Gawker out  but could the real reason have been its Epstein coverage? Here s some of what Gawker said about the Epstein case: Donald Trump, Courtney Love, former Israeli Prime Minister Ehud Barak, and uber-lawyer Alan Dershowitz may have been identified by a butler as potential  material witnesses  to pedophile billionaire Jeffrey Epstein s crimes against young girls, according to a copy of Epstein s little black book obtained by Gawker.An annotated copy of the address book, which also contains entries for Alec Baldwin, Ralph Fiennes, Griffin Dunne, New York Post gossip Richard Johnson, Ted Kennedy, David Koch, filmmaker Andrew Jarecki, and all manner of other people you might expect a billionaire to know, turned up in court proceedings after Epstein s former house manager Alfredo Rodriguez tried to sell it in 2009. About 50 of the entries, including those of many of Epstein s suspected victims and accomplices as well as Trump, Love, Barak, Dershowitz, and others, were circled by Rodriguez. (The existence of the book has been previously reported by the Daily Mail. Gawker is publishing it in full here for the first time; we have redacted addresses, telephone numbers, email addresses, and the last names of individuals who may have been underage victims.)According to an FBI affidavit, Rodriguez described the address book and the information contained within it as the  Holy Grail  or  Golden Nugget  to unraveling Epstein s sprawling child-sex network. Epstein reportedly also had additional ties to Trump, with both men having on and off again been implicated in a separate Jane Doe  sexual assault lawsuit that was recently dropped in the US court system, apparently for a third time since 1994.The Washington Times discusses how other high-level clients allegedly used Epstein s services   including a member of the British royal family. The following passage is from a report filed in January of 2015: And even post-prison, Mr. Epstein kept up some of these high-profile relationships   leading to his most recent emergence in the news, which involves allegations that he provided Prince Andrew with sexual services from a then-17-year-old American girl, Virginia Roberts.Ms. Roberts, now 30, has filed a federal lawsuit in Florida against Mr. Epstein, alleging he forced her to serve as a  sex slave  and paid her $15,000 to perform sexual services for the prince, the New York Post reported. Here s Sean Stone from RT s Watching the Hawks interviewing Conchita Sarnoff, the Executive Director from The Alliance to Rescue Victims of Trafficking to discuss Epstein s heinous past Another strange turn in the Clinton Email scandal came late in the election, only days before voters went to the polls. Suddenly, it was revealed by media outlets that due to the investigation into the sexting scandal involving disgraced former U.S. Representative Anthony Weiner, (husband of chief Clinton aide Huma Abedine) FBI director James Comey would announce that the agency would be reopening the Clinton email probe.Interestingly, according to a recent interview with Breitbart News, Hillary Clinton herself was alleged to have taken 6 trips to Epstein s Little Saint James island, according to private security kingpin and Blackwater founder Erik Prince. Prior to the election and right after the FBI reopened its investigation into the Clintons (before promptly being shut down for a second time), the controversial mercenary made the extraordinary claim.It s important for readers to note how explosive this revelation truly was, and yet, not one word from either the mainstream media, or  alternative media,  outside of Breitbart and few other blogs, on this potentially devastating allegation. Prince asserted that the NYPD found  way more stuff  within the 650,000 recently scrutinized emails. The news reportedly sent shock waves into the upper echelon of the Democratic Party and the Clinton campaign   as the NYPD was said to have been ready to hand out indictments  into additional criminal activity, possibly involving money laundering and underage sex, among other illicit activity.Prince s commentary on the Democratic Party s apparent corruption was released around the same time as a batch of Podesta Emails that sent internet sleuths down a pizzagate rabbit hole. Prince s claim then seem to be grafted onto the pizzagate theory by some YouTube commentators, only without any actual concrete evidence.In early 2016, the embattled Blackwater (renamed Academi, and then Xe) founder Prince, was the subject of an inquiry that alleged he was involved in a major money laundering scheme with the Bank of China in Libya. In 2014, Prince supposedly setup  Frontier Services Group in 2014 with the help of major Chinese investment firm Citic Group. When you factor in how Prince himself became the subject of a major scandal and is now on the offensive, it s hard to know whether his claims are true or just a political ploy aimed at deflecting his own controversy. However, his claims do offer a dark view into a political party that has been in a downward spiral for several years.Wheat from the ChaffWith all of that in mind, for any true investigation to take place, the facts still need to be separated from what might only be innuendo and overt speculation.Other apparent conspiracies were also brought into the fold during the ascendency of #pizzagate, as social media users seemed to make some startling connections, although without a complete story. The lack of coherence of pizzagate allegations opened the door for some of the usual suspects to come in to steer the conversation away from some potentially damaging connections back to the Clintons. Establishment gatekeeper publication The Daily Beast reported the following in the passage below: Written by user  PleadingtheYiff,  the post attempts to link the Clinton campaign to Laura Silsby, an Idaho missionary who served a six-month prison sentence in Haiti after the organization she founded, New Life Children s Refuge, attempted to transport 33 Haitian children to the Dominican Republic after the 2010 Haiti earthquake.The user s primary proof of links between the Clintons and Silsby:  Huma Abedin was constantly forwarding Hillary articles on this woman s organization.  The post provides no email sent from Clinton to Abedin regarding the incident.The theory then includes a major error. One of the first things Hillary did when she took over the scene in Haiti was to get Laura off the hook,  PleadingtheYiff writes, then links to a website called  AboveTopSecret.com. Silsby wasn t freed and instead remained incarcerated in Haiti, where she faced trial and was found guilty. The nine other missionaries jailed alongside her were freed. Is it possible that the Laura Silsby case was buried under endless collateral damage from pizzagate?With even more damning American conspiracies in 2016, such as the DNC fraud, violent protests funded by George Soros, or the Pentagon paying PR firm Bell Pottinger $540 million to produce fake terror videos, in addition to the Lolita Express revelations concerning the Clintons in particular, you have to wonder whether the pizzagate theory was an unintended coincidence or if it was yet another socially engineered agenda that masked other media stories.What s most obvious, however, is how the pizzagate story and spin-off controversies appear to have obfuscated other real criminality contained within the Podesta email leaks, while providing a media smoke screen for the conspiracies mentioned directly above.In the article below we ll take another look at the pizzagate phenomena (along with Comet Ping Pong s gunman story, in addition to one of Washington DC s most notorious scandals) and how the events surrounding the online conspiracy, could be used to shape new censorship protocols in the wake of US media s witch-hunt for  fake news  sites   LEAKED    John Podesta former chairman for Hillary Clinton s 2016 presidential campaign, pictured with Clinton chief aid Huma Abedin on the Clinton campaign jet. Spirit Cooking & #Pizzagate: A Media-Driven Psy-Op?Within the contents of the controversial Podesta dispatch by Wikileaks in early November, there appeared to be a distressing email chain between Tony Podesta, a top Democratic lobbyist of the Podesta Group (brother of John Podesta, Hillary Clinton s campaign chairman) and a New York City-based Serbian performance artist named Marina Abramovic.The 2015 email exchange between Tony Podesta and Marina Abramovic (image, left) drew a sharp outcry from users on social media platforms like Twitter and Reddit, after the email conversation revealed a dinner party involving a performance art-based ritual calling for the use of real human blood and bodily fluids, referred to as  spirit cooking.   While Abramovic s occult-influenced art could be seen as sick and deeply disturbing to many, and even degenerate depravity to some   it s important to emphasize there was nothing criminal about her email with Tony Podesta or her dark creations captured on video from as early as two decades ago.The origins of Abramovic s spirit cooking appeared to some researchers to be linked directly to Aleister Crowley s, Thelma religion. This prompted some alternative media outlets to link to Everpedia as their primary source to explain the  truth  behind spirit cooking. The internet-based conspiracy ultimately led to increased speculation, turbo-charging even more spurious connections.Another major magnet for pizzagate researchers was Abramovic s book published in 1997, the self-proclaimed  grandmother  of performance art released a book entitled  Spirit Cooking with essential aphrodisiac recipes,   a rare book that contained a sequence of etchings and perverted recipes.As theories about spirit cooking exploded on social media, a series of Podesta emails were leaked containing what had been thought to be coded language concealing an elite underground child sex ring, according some quasi-investigative online sites and mega forums, citing 4chan and Reddit posts.In addition, it appears that a series of emails released by Wikileaks, here and here, are partially what led to one man, James Alefantis, owner of Comet Ping Pong, to become the central pizzagate antagonist. The fact that Alefantis is listed as one of DC s top 50 most influential people   only fuelled speculation that the internet case was  a slam dunk. Social media inquisitors shared pictures from the Instagram account of Alefantis (now blocked to the public) and other friends and contacts linked with him, in an attempt to conclude a vast conspiracy. While many of these images appeared disturbing in the context of the two conspiracies mentioned above, the images themselves did not prove any actual crime had taken place.As the conspiracy deepened, symbols from the Washington DC restaurant/venue Comet Ping Pong (derived from its menu and logo, as well as other pizza restaurants), in addition to apparent coded Podesta emails, sparked an outcry on social media as the symbols used appeared to mimic those according to a Wikileaks release supposedly from the FBI. The apparent leaked release from the FBI disclosed certain markers in sex trafficking and abuse cases. While there might be similarities in the graphics observed at Comet Ping Pong and nearby Besta Pizza, the evidence is still purely circumstantial and certainly nothing conclusively linking the DC-based pizzerias to any abuse ring. Internet trawlers also claimed to have unearthed photos of the digging of a secret basement-dungeon underneath Comet Ping Pong   a claim which later turned out to be false.Note that this one planted detail in the pizzagate story   the evocative image a  secret basement  reserved for child torture and abuse   sent this story into orbit. This is one example of how one or two simple details injected into a storyline can inflame the narrative and motivate forum users beyond what any real forensic investigation could, by creating a virtual crusade.In fact, it was revealed that  Comet uses a low-slung refrigerator at Buck s Fishing & Camping, Mr. Alefantis s other restaurant two doors down on the same block, to store produce, cheese and proofing dough.   (See left photo by Cecilia Kang)It s worth noting that 4chan was also behind one of the most speculative claims surrounding MH370, a story that allegedly involved the patent holders of the tech company Freescale Semiconductor which turned out to be false. Conversely, 4chan has provided some valuable crowd-sourced information, like in the wake of the 2013 Boston Bombing when 4chan users posted a number of photos indicating the presence of other operatives involved in the event   a point totally blacked-out by US authorities and media.Interestingly, the 4chan website is said to be  struggling to survive  financially in part because of increased users driven by viral and clickbait-like material like pizzagate.Through hyper-speculative and often misleading information spread by social media users, the pizzagate narrative took shape Media TripwireAs the US media brushed off the perceived content of the Podesta emails mentioned above, a ground swell of speculation, hyperbole and unsubstantiated claims filled the vacuum online, as enthusiasm continued unabated.It s also important to recognize that one of the main propellers of the initial Pizzagate story was a popular disinformation website called Your News Wire, who regularly feed the public with completely fake news stories online which get propagated through social media. This website has successfully managed to fool many online readers because it will regularly mix real news articles in with its many invented stories. A viral fake story like this would have netted  Your News Wire  tens of thousands of dollars in ad click revenue.BuzzFeed reports: On Oct. 30, a white supremacy Twitter account that presents itself as belonging to a Jewish lawyer in New York tweeted that the NYPD was looking into evidence that emails from Anthony Weiner s laptop contained evidence of Clinton involvement in an  international child enslavement ring. The next day, Oct. 31, Sean Adl-Tabatabai, who used to work with British conspiracy theorist David Icke, took note of the message board post and the tweet. He wrote a post for his site, YourNewsWire.com. His story took another step by claiming an  FBI insider  had confirmed the claims. Where did this this new FBI source come from? Adl-Tabatabai cited a thread on a 4chan message board from back in July. That thread started when an anonymous user invited questions after describing themself as  a person with intimate knowledge of the inner workings of the Clinton case.  (Adl-Tabatabai did not respond to questions sent to him by BuzzFeed News.) By this point, the story became a viral ball of fire, and there was no stopping it.You have to wonder if the spirit cooking/pizzagate emails, might have been used as a tripwire by the establishment to initiate increased internet censorship, a story that has gathered a big boost from highly speculative ideas pasted together by various social media personalities on YouTube and then codified by large popular  alt-media  outlets like Infowars.com here and here, who levied unconfirmed allegations at Comet Ping Pong restaurant and other private businesses in an affluent area of Washington DC.Few conspiracies in recent years have generated as much public outrage, scrutiny and intense speculation. However, we should question the timing of this conspiracy as it relates to the cyber censorship on the table following the  fake news  crisis.The BBC stated the following after a Podesta email thread leaked in November, mentioned the Alefantis connection: In early November, as Wikileaks steadily released piles of emails from Clinton s campaign chairman John Podesta, one contact caught the attention of prankster sites and people on the paranoid fringes.James Alefantis is the owner of Comet Ping Pong, a pizza restaurant in Washington. He s also a big Democratic Party supporter and raised money for both Barack Obama and Hillary Clinton. He was once in a relationship with David Brock, an influential liberal operative.Alefantis   who s never met Clinton   appeared in the Podesta emails in connection with the fundraisers. Other claims attached to the pizzagate theory included a link to a tweet from the deceased investigative reporter, Andrew Breitbart. The pizzagate theory seemed to gather steam from this screenshot of an alleged Breitbart tweet below   but is there more to the story?Upon reviewing the origins of the Breitbart tweet now intertwined with the pizzagate theory, there appears to be some contention about what Breitbart might have been referencing. Here s the political blog Hot Air on the matter: It s a classic Andrew Tweet. His abbreviations, his invention of words ( cover-upperer ) tells me that this was probably Andrew s work. I never really knew him to allow anyone to Tweet on his behalf but I suppose it s possible at some point he did so without my knowledge. So, for our discussion let s stipulate that it was Andrew who sent this out. What is he talking about? Anyone who knew Andrew well and worked closely in his sphere and knew what he would talk and Tweet about with regard to  underage sex slave(s)  and John Podesta, knows exactly what he is referring to: ACORN If you don t remember James O Keefe, Hanna Giles and the ACORN story that broke on Andrew s Big Government website in 2009, here s a quick description via Wikipedia, which has a lot of bogus takes on the story, but this brief description is correct: In July and August 2009, Giles and O Keefe visited ACORN offices in Baltimore, Washington, D.C., Brooklyn, San Bernardino, San Diego, Philadelphia, Los Angeles, and Miami. Giles dressed as a prostitute, while O Keefe wore white khakis with a blue dress shirt and/or tie and claimed to be her boyfriend. Giles and O Keefe recorded the encounters using hidden cameras and pretended to be seeking advice on how to run an illegal business that included the use of underage girls in the sex trade. The article goes on to state that the ACORN saga, which had been instrumental in Breitbart s rise in media, had been in some aspect involved an underage abuse story due to the undercover work of James O keefe and Hannah Giles.  STRANGE DAYS    Comet Ping Pong resides in a quiet affluent DC neighborhood. (Image Source: chicago sun-times)The Comet Ping Pong GunmanIn early December, 28 year-old actor Edgar Maddison Welch was arrested by authorities after reportedly discharging a rifle inside Alefantis s Comet Ping Pong restaurant. Welch had been apparently  self-investigating  Comet Ping Pong, the pizzeria at the epicenter of a shocking conspiracy asserting that the establishment is somehow allegedly involved in an expansive child trafficking abuse ring.Welch s background may have raised a few eyebrows of those who have chronicled the profiles of various gunman in recent years (such as his acting/film production past documented on IMDB), he was also recently involved in a major traffic accident in October, badly injuring a young teenager, potentially impacting his state of mind, as well as being exposed in a potential criminal case.While 21WIRE has outlined the possibility of informants being used in shooting events in the past   usually those individuals have had a lengthy criminal past with multiple links to government, military and sometimes the entertainment industry   Welch doesn t entirely fit this typical mold of a convict turned informant or agent provocateur.Perhaps there s another answer The online site Heavy.com released the following information regarding Welch s father and his work preventing child abductions: Harry Welch Jr. is a former police officer, firearms instructor and expert witness on firearms, according to his IMDB profile. He also writes that his family has hosted foster children and has worked to prevent child abuse.He was,  elected President of the North Carolina Crime Stoppers Association (two terms). Served as Executive Director for Protect-A-Child, a national, non-profit organization to prevent abuse and abduction of children. Appointed by Governor Jim Martin to the Governor s Commission on Child Victimization. Was Welch under psychological duress following his recent car accident, feeding into other aspects of his life, such as his father s work to prevent child abuse, then motivated into action by an online conspiracy?Although it s difficult to accurately gauge Welch s true motivation, some YouTube personalities absolutely contend that his involvement was scripted    to smear the a new burgeoning  pizzagate research community  online. Additionally, some researchers cite Welch s relief visit to Haiti in 2010, as absolute  proof of a hoax  because alleged Clinton Foundation ties to human trafficking in Haiti   claims that have to this date, have yet to be completely proven.This part of the pizzagate conspiracy gained steam when Monica Petersen, an activist for sex-worker rights, who over the summer was working for  Claude d Estree, the executive director of the Human Trafficking Center at the University of Denver,  apparently died of a suicide in mid November while in Haiti. However, at the time the circumstances of her death were not entirely clear, which led to intense speculation not grounded in fact. If there was evidence of foul play, certainly this could be investigated as part of a possible cover-up, especially during such a hotly contested US election involving the Clintons   but until that issue is resolved, any talk of  proof  of a wider conspiracy is still just that, talk.The Petersen tragedy was then grafted onto the pizzagate story The Washington Post reported the following, according to executive director d Estree:  Petersen had gone to Haiti a number of times, but she was not there to research human trafficking and was not investigating the Clinton Foundation. He said she had been a  brilliant  research fellow, working the past two years to produce a still unpublished article that critiqued a controversial book on the business of sex trafficking by Siddharth Kara. Unfortunately, Petersen s story got packaged with allegations of a Clinton trafficking conspiracy that was then interwoven within the pizzagate theory and on the same day Welch walked into Comet Ping Pong, the widely-known site Infowars.com  posted a YouTube video titled  Haiti PIZZAGATE: Death of Child Trafficking Investigator,  without actually confirming the full details surrounding Petersen s death. Everpedia, one of the first websites to explain the details of spirit cooking, appeared to also fan the flames of an alleged conspiracy surrounding Petersen s death.An ABC News release below disclosed the full details regarding Welch s apparent criminal activity at Comet Ping Pong: At around 3 p.m., he arrived at the restaurant, where witnesses say they saw Welch holding an assault rifle across his chest, according to the complaint. As he moved toward the back of the restaurant, employees and customers fled the building.As everyone was fleeing, Welch encountered an employee returning from the back freezer, according to court documents. The employee said Welch pointed the AR-15 rifle in his direction, the documents state, but Welch denied raising the firearm at the employee.After the customers and employees left the restaurant, police created a perimeter around it, and at about 3:24 p.m. Welch exited the building with his hands in the air, complying with officers. The story continued, with the following description: At one point he said he encountered a locked door and attempted to open it with a butter knife, the complaint says. When that didn t work, he fired shots at the lock with his AR-15, documents say. Here s a screen shot of Welch s incident report MSM Fake News Witch-HuntFor the mainstream media, pizzagate was the gift that just kept on giving.Following Welch s encounter inside Comet Ping Pong, the conspiracy known as  pizzagate  has fallen into the manipulative hands of the corporate mainstream media, now castigating  speculative alternative media outlets,  which in-turn gave legs to the already convoluted  fake news witch-hunt being waged by mainstream media   specifically aimed at independent outlets and blogs that regularly produce insightful critical analysis.Additionally, since the spirit cooking and pizzagate theories were merging, there has been no solid evidence linking the restaurant to anything criminal, yet the theories continue to persist online, implicating private citizens in what is at this point a  victimless  crime. This has made alternative media an easy target for mainstream media and politicians calling for more  control  over social media content to help stop waht mainstream pundits were now describing as pizzagate s own campaign online abuse and  internet bullying  against Comet Ping Pong s owner and staff.From a legal stand point, the pizzagate conspiracy could also potentially lead to a court case against the root cause of certain defamatory allegations and apparent death threats   as Alefantis and other shop owners are private citizens and not public figures. This is something to consider when presenting any possible theory directly implicating an individual in a major felony. Unless there exists a real forensic smoking gun, or a whistleblower willing to come forward then there is no  case    only speculation which could be construed as libel and defamation without any supporting evidence.While the internet has zeroed-in on pizzeria-based conspiracies over the past two months, it s important to remember that Washington has had a long dark history linked to illicit prostitution and sex trafficking   including at least two such expansive cases that reached all the way to the White House   DISGRACED    Former US Congressman Barney Frank. Frank did not seek re-lection in 2012 after he was found to have engineered a $12 million TARP bailout for OneUnited Bank. (Image Source: businessinsider.com)Washington s Silence: Congressman Frank s  Call Boys In 2011, Democratic Congressman Barney Frank, decided not to seek re-election. While much of the mainstream media waxed poetic about his 30-year career, there was virtually no mention of his involvement in one of the most notable DC scandals over the past three decades.In 1989, it was discovered that Frank s then boyfriend Stephen Gobie (who the US representative once solicited for an $80 dollar sexual encounter) had been running a prostitution ring out of Congressman Frank s Capitol Hill apartment   according to Gobie, Barney Frank was fully aware of the operation.Here s a passage from a Business Insider article from 2011 that describes one of DC s darkest scandals that is barely mentioned by media outlets to this day: Reuters didn t mention it. The New York Times didn t bring it up. The Guardian didn t note it either. The Washington Post got to it in the 17th paragraph.In 1989 it was discovered that Barney Frank s boyfriend, Stephen Gobie, whom Frank had once hired as a male prostitute, was running a male-brothel out of the Congressman s home. Frank claimed he did not know about the prostitution ring in his home, but he did use the power of his office to  fix  33 tickets for Gobie. And he knowingly wrote a misleading letter to Gobie s probation officer in Virginia. Frank received a  reprimand  for fixing the tickets. Gobie maintained that Frank knew about the prostitution ring operation in his home. Even writers who were absolutely untroubled by the openly homosexual lawmaker in 1989 thought that Frank should have resigned. Or they at least acknowledged that there was an abuse of his office. Additionally, the Washington Post reported that Gobie  maintained a relationship with Gabriel A. Massaro, the principal of Chevy Chase Elementary School, and used an office at the school in late 1987 to make telephone calls and have one client meet a prostitute. While Massaro was placed on administrative leave, the conspiracy deepened as the Washington Post followed up the breaking story providing the following details: With Congress in recess, it s too soon to tell if the Massachusetts Democrat will weather his admission that he paid Gobie for sex, hired him as a personal aide and wrote letters on congressional stationary to the convicted felon s probation officer. Shockingly, the scandal continued to persist as the investigation led to other political inquiries. Frank s scandal appeared to overlap and dovetail with another disturbing ring that reached all the way to the White House during the Reagan-Bush era.The Franklin Cover-upThe Barney Frank scandal indeed followed another appalling and chilling scandal dubbed the  Franklin Cover-up.  The shocking case involving Omaha, Nebraska s Franklin Credit Union employee named Lawrence King, who looted his employer to the tune of $38 million dollars. Here s a Washington Post article from 1990 on the matter: Lawrence E. King Jr., everyone now agrees, had a remarkable knack for stretching a dollar. On a salary of $16,200 a year, the credit union manager drove a $70,000 white Mercedes   and still could afford to spend $10,000 a month on limousines. His credit card charges topped $1 million, he owned a four-story house on 26 acres overlooking the Missouri River, and his floral bill alone came to $146,000 during a fragrant, 13-month period in 1987-88. While King s embezzlement was troublesome, it paled into comparison to a wider charge of trafficking and abuse levied at the credit union manager. The Post continued, outlining the awful truth about the abuse tied to King, in a group called the Franklin Five: In June 1988, a social worker at a mental hospital in Omaha reported allegations of possible child abuse to the Nebraska Foster Care Review Board, an agency that monitors the placement of several thousand children in the state.The board subsequently asked Omaha police and the state attorney general to investigate possible  child exploitation,  a  child prostitution ring  and  inappropriate activities by Larry King. More evidence in the same report revealed the following: 21 hours of videotaped testimony from three new witnesses, all in their early twenties, who said that, as minors, they had been victims of physical and sexual abuse. The book entitled The Franklin Cover-up, by former Nebraska state Senator and lawyer, John De Camp, chronicled a case that implicated politicians, judges, those in the military, police, as well as members of the intelligence community in the subsequent abduction of numerous children over the course of many years.Lawrence King was eventually convicted on charges of theft from the Franklin Credit Union and ordered to pay damages to Paul Bonacci for the apparent abuse he suffered during his abduction. Bonacci later told investigators he had participated in the abduction of Johnny Gosch, but according to (Polk County Attorney John Sarcone ) The Des Moines Resgister,  Bonacci has said he took part in Johnny Gosch s abduction, but Sarcone said investigators concluded any involvement by Bonacci was not credible. Here s a short summary of the disturbing case below: On February 5, 1999, in U.S. District Court in Lincoln, Nebraska, an extraordinary hearing occurred in Paul A. Bonacci v. Lawrence E. King, a civil action in which the plaintiff charged that he had been ritualistically abused by the defendant, as part of a nationwide pedophile ring linked to powerful political figures in Washington and to elements of the U.S. military and intelligence establishment. Three weeks later, on February 27, Judge Warren K. Urbom ordered King, who is currently in Federal prison, to pay $1 million in damages to Bonacci, in what Bonacci s attorney John DeCamp said was a clear signal that  the evidence presented was credible. The Frank and Franklin conspiracies outlined above sent shock waves through the establishment in Washington, and while it is possible we are seeing another trafficking cover-up happening again today, one should consider that thus far, there have been no victims found to have been abused regarding Comet Ping Pong or any other business wrapped up within the online pizzagate theory.A Franklin Cover-up documentary called  Conspiracy of Silence  was made and allegedly pulled from circulation following the fallout from a case that also implicated the Catholic founded non-profit orphanage, Boys Town.In a related scandal exposed in 2002, The Boston Globe s Spotlight Team  first revealed that the leader of the Catholic Church in Boston, Cardinal Bernard Law, allowed one of his priests to continue working even though Law knew that Fr. John Geoghan had spent his career sexually assaulting children. The story became so huge, that it led to more than 250 priests being publicly named in abuse allegations, covering some 100 cities in the US and another 100 globally. In 2015, Hollywood released the film Spotlight, which chronicled the tales of abuse.Similarly in the UK, over the past few years, an investigation into the Jimmy Savile abuse scandal  was compounded by BBC s own cover-up of the serious claims. The Savile scandal and cover-up featured many whistleblowers and victims coming forward, and also showed a high degree of cover-up at the police department level as well.While observing the connective aspects of various scandals, DC s political scene has also revealed some telling business and establishment links which raise even more interesting questions   MEDIA MACHINE    Democratic political operative David Brock. (Image Source: twitter)There are some interesting aspects of pizzagate which have been almost untouched by internet sleuths.Below we ll take a look at George Soros-funded Media Matters and its director David Brock and examine his past connections and tangential links to both Barney Frank and James Alefantis Media Matters  & Political PlotsIn 1997, political operative David Brock was once a right-wing media hit-man so to speak, writing conservative op-ed s taking on college campus  marxists  and was an investigative reporter for the weekly news magazine Insight on the News (published by The Washington Times). Some time later, Brock became a Democratic political operative and Soros righthand man at Media Matters.Some reports also claim that Brock (seen on the left with Barney Frank) joined on with Sidney Blumenthal and the Clintons simply over money   but others suggest a different tale   one that took place during the course of Brock completing a somewhat strange book entitled  The Seduction of Hillary Clinton,  which deflected controversies aimed at the Clintons and took a softball approach to Hillary s political evolution.Over the years, many critics have questioned Brock s hidden Washington past and his unlikely political evolution Blackmail? Brock, Grey & AlefantisIn 2012, FOX NEWS revealed the following, as Media Matters chief David Brock paid a former domestic partner $850,000 after being threatened with damaging information involving the organization s donors and the IRS   a deal that Brock later characterized as a blackmail payment, according to legal documents obtained by FoxNews.com. Brock s bitter separation from William Grey led to accusations of blackmail and harassment: In an acrimonious lawsuit settled at the end of last year, Brock accused William Grey of making repeated threats to expose him to the  scorn or ridicule of his employees, donors and the press in demanding money and property.  Brock claimed in legal papers that he sold a Rehoboth Beach, Del., home he once shared with Grey in order to meet Grey s demands, which he called  blackmail  in the lawsuit. Brock s legal trouble with Grey appeared to be the result of a severed relationship after 10 years, with Brock ending the relationship, he began dating Washington, D.C., restaurant impresario James Alefantis,  according to reports.Considering how Brock and Grey s end played out according to court records revealed by the media, one might consider that this latest DC drama unfolding in the form of pizzagate, could also be the result of bad-jacketing (as suggested by the blog auticulture) after a bitter love triangle involving the trio   in the lead up to one of the most contentious US presidential elections in history.According to Ward Churchill s,  Agents of Repression: The FBI s Secret Wars Against the Black Panther Party and the American Indian Movement: Snitch-jacketing or bad-jacketing refers to the practice creating suspicion   through the spread of rumors, manufacture of evidence etc Bad-jacketing had been a technique employed by the FBI under their program COINTELPRO.Domestically in America, the FBI created the counter-intelligence program known as COINTELPRO, to influence, disrupt and coerce political factions from the inside out, infiltrating countless political groups and movements. According to reports these included the following,  The Black Panther Party, The Communist Party of America, the Ku Klux Klan, the Socialist Workers Party, the New Left, the Students for a Democratic Society, the American Indian Movement, the Chicano Movement, the Puerto Rican Liberation Movement, Communist groups, anti-war organizations, Hollywood stars sympathetic to these groups, and civil rights leaders. Between 1956 and 1971 the FBI s controversial program influenced and radicalized hundreds of left-wing and right-wing groups to control and neutralize political dissidents across America. No part of politics was immune to COINTELPRO.Is it possible that a DC love triangle that once involved Brock, Grey & Alefantis, led to a PR-driven form (as a part of Brock s Correct the Record or something else) of bad-jacketing which then cascaded through the Democratic Party elite, ultimately taking its triangle shape   through the use of pizzagate and others? MEDIA CONTROL    (Photo illustration redesigned by 21WIRE s Shawn Helton)Censorship & the  Fake News  Witch-HuntIn early November, just prior to the US presidential election, CNN published an article stating the following: A recent BuzzFeed study of  hyperpartisan Facebook pages  found that these pages  are consistently feeding their millions of followers false or misleading information. Three weeks later in a republished article written by Robert Parry from Consortium News, it was disclosed that major media companies had been heavily involved in pushing the fake news meme, citing many election-based posts made online as the culprit: In its lead editorial on Sunday, The New York Times decried what it deemed  The Digital Virus Called Fake News  and called for Internet censorship to counter this alleged problem, taking particular aim at Facebook founder Mark Zuckerberg for letting  liars and con artists hijack his platform. Below is a passage from a recent report filed at 21WIRE discussing the fake news witch-hunt, which has now led to new calls for internet censorship: Over the past month, The Washington Post published an irresponsible propaganda piece that blamed the 2016 US election results on  fake news  dispensed via sophisticated Russian propaganda. These accusations of course were completely unfounded, deliberately feeding into a paranoid trial by media format that is no doubt echoed by US government sock puppet accounts that act as a digital firing squad by redirecting public perception.While the US mainstream media contends that  fake news  somehow altered the 2016 presidential election cycle, it s important to remember that the US government has some pedigree here, having been behind the creation of bogus social media accounts and personalities for years now.Back in 2011, The UK s Guardian reported that the US government via CENTCOM, was looking to gain a foothold over information shared on the internet through fake online personal accounts: The discovery that the US military is developing false online personalities   known to users of social media as  sock puppets    could also encourage other governments, private companies and non-government organisations to do the same.The Centcom contract stipulates that each fake online persona must have a convincing background, history and supporting details, and that up to 50 US-based controllers should be able to operate false identities from their workstations  without fear of being discovered by sophisticated adversaries . This is just one example. Who really knows how widespread this practice is.You can expect a litany of allegations to continue from the mainstream, along with their faux war on fake news sites which has been used deliberately to discredit and frame genuine investigative analysis in alternative media, its intended target. Western media s anti-Russian hyperbole has morphed and broadened somewhat, thanks to Facebook, Google and Twitter and the alphabet establishment news agencies, Washington has a new go-to scapegoat joining the tired old Russian  hack  meme   independent media. SEE ALSO: Washington Post Sloppy  Journalism  Blames Russia for  Fake News  Crisis and Trump s Win, While Pushing Neo-McCarthyismIt appears as though there has been a coordinated effort to combine the latest run of conspiracies in America, whether it s the fake news meme, the  Russian election-hack  meme or pizzagate    they all seem to be have been used to undermine key aspects of internet freedom.Below is an interview with James Alefantis featured on Megyn Kelly s Kelly File discussing the highly speculative pizzagate theory. During Kelly s interview, the focus centered around unsubstantiated claims levied at the business owner and his place of business Here s a very detailed analysis provided by researcher David Icke discussing the multi-layered theories wrapped up in the pizzagate conspiracy and its relation to censorship and so-called  fake news In ConclusionThe overtly conspiracy-tinged pizzagate theory, emerged like a hydra out of sub-Reddit and 4chan threads and Twitter posts, while catapulting only the most sensational aspects into the mainstream matrix. The story conjures many Hollywood and government tales of cover-up, murder, abuse and deception throughout history.The perceived presence of a politically charged Satanic sex-cult intertwined within a paedophilic abuse ring, seemed to have only enhanced the narrative for many internet-sleuths, who in some cases, held steadfast to a biased guilt-by-association meme   even in the absence of any known victims, and without any forensic evidence or any eye-witness testimony of a crime.This is not to suggest that VIP child sex rings and ritual abuse cases do not exist in America. One such murder spree, known as the  Son of Sam  case, was reopened in part due to evidence of Satanic cult activity and multiple offenders. Although the findings were suspended, the case was never officially closed.While people should certainly have the ability to express their opinion about pizzagate without shame or ridicule, using biased, misleading or fraudulent information (in some cases dispensing defamatory or threatening remarks) in a rapid-fire echo chamber, could ultimately undermine any legitimate investigation if there truly was a crime committed here, or elsewhere. If that is the ultimate legacy of pizzagate, then it is a devastating self-inflicted wound for the alternative media.While it s true that sites like Reddit and Twitter could cite the 1996 Communications Decency Act, as a protection for hosting defamatory information posts by users, there could be real consequences for those deliberately posting defamatory content with no real evidence to substantiate such allegations.Another risk with a whirlwind like this, is that it would be very easy for an agent provocateur to post a slick YouTube video or craft an article   with the intention of whipping-up a storm that could be used to damage independent media as a whole. It s very possible that this has already happened at some point over the life of the internet, and it s certain that the powers that be will use this tactic again in the future.Above all however, it s important to separate the fallout from the pizzagate story and other fake news allegations, as those in mainstream media such as The New York Times, The Washington Post and the self-anointed  fact checker Snopes.com, seem to have propped up this pizzeria conspiracy as a battle cry to introduce the idea of new internet censorship protocols.It s anybody s guess what will be the final outcome of the #Pizzagate phenomenon, but for the moment, it seems to have collapsed under the weight of its own mountain of digital ash. Maybe that was the idea. READ MORE MSM LIES AT: 21st Century Wire MSM FilesSUPPORT 21WIRE  SUBSCRIBE NOW & BECOME A MEMBER @21WIRE.TV'

Tokenize the sample news stories

In [20]:
g_tok = model_helper_functions.tokenize_and_normalize_title_and_text('', g)

In [21]:
r_tok = model_helper_functions.tokenize_and_normalize_title_and_text('', r)

In [22]:
r2_tok = model_helper_functions.tokenize_and_normalize_title_and_text('', r2)

In [23]:
a_tok = model_helper_functions.tokenize_and_normalize_title_and_text('', a)

In [24]:
b_tok = model_helper_functions.tokenize_and_normalize_title_and_text('', b)

In [25]:
bb_tok = model_helper_functions.tokenize_and_normalize_title_and_text(bb_title, bb_text)

In [26]:
nyt_tok = model_helper_functions.tokenize_and_normalize_title_and_text(nyt_title, nyt_text)

In [27]:
nyt_tok_2 = model_helper_functions.tokenize_and_normalize_title_and_text(nyt_title_2, nyt_text_2)

# Predict Classes

In [28]:
pipeline.predict([g_tok, r_tok, r2_tok, a_tok, nyt_tok, nyt_tok_2, # True
                  b_tok, bb_tok]) # Fake

array([0, 1, 1, 1, 1, 1, 0, 0])

The expected outcome was [1, 1, 1, 1, 1, 0, 0]

The three Reuters stories (2 new and one from the dataset) and the two from the NYT were classified correctly, but interestingly the Guardian story was not even though there are now Guardian stories in the dataset.

The two fake news stories (1 new and 1 from the dataset) were classified correctly.

# Predict Class of Full set of Guardian politics articles

## Load Data

In [29]:
df = pd.read_csv(DATA_PATH + PRE_PROCESSED_DATA_FILE_NAME, 
                 low_memory = False)

In [30]:
df.tt_tokens = df.tt_tokens.map(ast.literal_eval)

## Do Predictions

In [31]:
results = pipeline.predict(df.tt_tokens)

In [32]:
np.unique(results, return_counts=True)

(array([0, 1]), array([ 1945, 10649]))

# Summary

The classification results on the full set of Guardian politics articles is much better this time returning an accuracy on the set of 85% that is only a little bit lower than the accuracy of the model test set, 89%.